<p align="center">
  <img width="550" height="281" src="http://encuentratubeca.mx/wp-content/uploads/2019/04/logoudgvectores_1_origsdaasss.png">
</p>

# <center>Proyecto Final de la materia Desarrollo de Proyecto II</center>             
------
## Maestría en Ciencia de los Datos
#### Alberto Martínez Lara
#### Mauricio Arnoldo Tenorio Vargas

#### Mayo 2023

In [1]:
# Importamos librerias necesarias para el analisis de los datos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import altair as alt
import nltk

In [26]:
alt.renderers.enable('mimetype')

RendererRegistry.enable('mimetype')

#### Informacion del Dataset
Dataset utilizando 

Variables:

* text: Contenido del tweet.
* favorited: .
* favoriteCount: Conteo de veces que el tweet fue marcado con favorito por otro usuario.
* replyToSN: Contenido del tweet, r.
* created: Fecha, hora y minuto de creación del tweet.
* truncated: Indicador de si el Tweet fue truncado al momento de obtenerlo con el API.
* replyToSID: .
* id: Identificador del tweet.
* replyToUID: .
* statusSource: html source.
* screenName: Usuario.
* retweetCount: Numero de retweets.
* isRetweet: Indicador de si el tweet es un Retweet o Tweet regular..
* retweeted: Indicador de si el tweet fue retweeteado.
* longitude: longitud del origen del tweet.
* latitude: latitud del origen del tweet.

In [3]:
#Descargamos dataset directamente desde el url
path = "https://raw.githubusercontent.com/AlbertoMCD/ChatGPT_tweets/main/data/GPT.csv"
df = pd.read_csv(path)

In [4]:
df.head()

,text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted,longitude,latitude
0,RT @gdb: ChatGPT as a tool for Congress: https...,False,0,NaN,4/27/2023 17:35:29,False,NaN,1651641228520194048,NaN,"<a href=""http://twitter.com/download/iphone"" r...",nos_ult,58,True,False,NaN,NaN
1,RT @gdb: ChatGPT as a tool for Congress: https...,False,0,NaN,4/27/2023 17:35:28,False,NaN,1651641223428317185,NaN,"<a href=""http://twitter.com/download/android"" ...",verobouvier,58,True,False,NaN,NaN
2,RT @moemicky18: So chatgpt failed an accountin...,False,0,NaN,4/27/2023 17:35:27,False,NaN,1651641221893308416,NaN,"<a href=""http://twitter.com/download/iphone"" r...",uzayrmug,2,True,False,NaN,NaN
3,RT @JeanPROFITbiz: Les gens gagnent des millie...,False,0,NaN,4/27/2023 17:35:27,False,NaN,1651641218059608094,NaN,"<a href=""http://twitter.com/download/iphone"" r...",SachaMery,359,True,False,NaN,NaN
4,"does chatgpt mean I, as a wet lab biologist wh...",False,0,NaN,4/27/2023 17:35:26,True,NaN,1651641214557364234,NaN,"<a href=""http://twitter.com/download/android"" ...",ruth_hook_,0,False,False,NaN,NaN


In [5]:
df.shape

(3000, 16)

Elminaremos variables inecesarias para el analisis.

In [6]:
df = df.drop(['replyToSID', 'id', 'replyToUID', 'longitude', 'latitude', 'replyToSN'], axis=1)

In [7]:
df.shape

(3000, 10)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text           3000 non-null   object
 1   favorited      3000 non-null   bool  
 2   favoriteCount  3000 non-null   int64 
 3   created        3000 non-null   object
 4   truncated      3000 non-null   bool  
 5   statusSource   3000 non-null   object
 6   screenName     3000 non-null   object
 7   retweetCount   3000 non-null   int64 
 8   isRetweet      3000 non-null   bool  
 9   retweeted      3000 non-null   bool  
dtypes: bool(4), int64(2), object(4)
memory usage: 152.5+ KB


### EDA & Data Cleaning

In [9]:
df.isnull().sum()

text             0
favorited        0
favoriteCount    0
created          0
truncated        0
statusSource     0
screenName       0
retweetCount     0
isRetweet        0
retweeted        0
dtype: int64

In [37]:
alt.Chart(df).mark_arc().encode(
    theta = alt.Theta(field='count(favorited):Q',type='quantitative'),
    color=alt.Color(field='favorited', type="nominal"),
)

Unable to obtain driver using Selenium Manager: Selenium Manager failed for: C:\Users\betop\anaconda3\lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe --browser chrome --output json.
request or response body error: error reading a body from connection: connection reset
Unable to obtain driver using Selenium Manager: Selenium Manager failed for: C:\Users\betop\anaconda3\lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe --browser firefox --output json.
request or response body error: error reading a body from connection: connection reset


ValueError: No enabled saver found that supports format='png'

alt.Chart(...)

There are only "FALSE" values in the favorited column, well proceed to delete that col from the dataframe.

In [11]:
df = df.drop(['favorited'], axis=1)

Distribution by Hour created

In [12]:
df['createdHour'] = df.created.str[9:12]
df.createdHour

0        17
1        17
2        17
3        17
4        17
       ... 
2995     16
2996     16
2997     16
2998     16
2999     16
Name: createdHour, Length: 3000, dtype: object

In [33]:
alt.renderers.enable("mimetype")
alt.Chart(df).mark_bar().encode(
    y = 'createdHour',
    x = 'count(createdHour):Q'
)

alt.Chart(...)

### Preprocesamiento de texto

#### 1. Extraer columna con texto del tweet

In [14]:
df_texto = df['text']
df_texto = pd.DataFrame(df_texto)
df_texto

,text
0,RT @gdb: ChatGPT as a tool for Congress: https...
1,RT @gdb: ChatGPT as a tool for Congress: https...
2,RT @moemicky18: So chatgpt failed an accountin...
3,RT @JeanPROFITbiz: Les gens gagnent des millie...
4,"does chatgpt mean I, as a wet lab biologist wh..."
...,...
2995,RT @arabtribunenews: د. فراس الحربي يكتب: ثورة...
2996,I actually had to redo this one twice because ...
2997,RT @rodtrent: Can ChatGPT work with your enter...
2998,RT @rowancheung: 2. Boston Dynamics puts ChatG...


#### 2. Detectar idioma original y traducir al inglés

In [15]:
from langdetect import detect

for index, row in df_texto['text'].iteritems():
    try:
        lang = detect(row) #detecting each row
        df_texto.loc[index, 'idioma'] = lang
    except:
        continue

df_texto

,text,idioma
0,RT @gdb: ChatGPT as a tool for Congress: https...,en
1,RT @gdb: ChatGPT as a tool for Congress: https...,en
2,RT @moemicky18: So chatgpt failed an accountin...,en
3,RT @JeanPROFITbiz: Les gens gagnent des millie...,fr
4,"does chatgpt mean I, as a wet lab biologist wh...",en
...,...,...
2995,RT @arabtribunenews: د. فراس الحربي يكتب: ثورة...,ar
2996,I actually had to redo this one twice because ...,en
2997,RT @rodtrent: Can ChatGPT work with your enter...,en
2998,RT @rowancheung: 2. Boston Dynamics puts ChatG...,en


In [16]:
from deep_translator import GoogleTranslator

for index, row in df_texto['text'].iteritems():
    try:
        translation = GoogleTranslator(source='auto', target='en').translate(row) #detecting each row
        df_texto.loc[index, 'text_ingles'] = translation
    except:
        continue

df_texto

,text,idioma,text_ingles
0,RT @gdb: ChatGPT as a tool for Congress: https...,en,RT @gdb: ChatGPT as a tool for Congress: https...
1,RT @gdb: ChatGPT as a tool for Congress: https...,en,RT @gdb: ChatGPT as a tool for Congress: https...
2,RT @moemicky18: So chatgpt failed an accountin...,en,RT @moemicky18: So chatgpt failed an accountin...
3,RT @JeanPROFITbiz: Les gens gagnent des millie...,fr,RT @JeanPROFITbiz: People are making thousands...
4,"does chatgpt mean I, as a wet lab biologist wh...",en,"does chatgpt mean I, as a wet lab biologist wh..."
...,...,...,...
2995,RT @arabtribunenews: د. فراس الحربي يكتب: ثورة...,ar,RT @arabtribunenews: Dr. Firas Al-Harbi writes...
2996,I actually had to redo this one twice because ...,en,I actually had to redo this one twice because ...
2997,RT @rodtrent: Can ChatGPT work with your enter...,en,RT @rodtrent: Can ChatGPT work with your enter...
2998,RT @rowancheung: 2. Boston Dynamics puts ChatG...,en,RT @rowancheung: 2. Boston Dynamics puts ChatG...


#### 3. Eliminar signos de puntuación
Con excepción de # y @ para identificar hashtags y cuentas.

In [17]:
import string
signos_puntuacion = string.punctuation
signos_puntuacion = signos_puntuacion.replace("#", "")  # quitar '#' de hashtags 
signos_puntuacion = signos_puntuacion.replace("@", "")  # quitar '@' de cuentas
signos_puntuacion

'!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~'

In [18]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in signos_puntuacion])
    return punctuationfree

#almacenar texto sin signos de puntuación
df_texto['text_clean'] = df_texto['text_ingles'].apply(lambda x:remove_punctuation(x))
df_texto.head()

,text,idioma,text_ingles,text_clean
0,RT @gdb: ChatGPT as a tool for Congress: https...,en,RT @gdb: ChatGPT as a tool for Congress: https...,RT @gdb ChatGPT as a tool for Congress httpstc...
1,RT @gdb: ChatGPT as a tool for Congress: https...,en,RT @gdb: ChatGPT as a tool for Congress: https...,RT @gdb ChatGPT as a tool for Congress httpstc...
2,RT @moemicky18: So chatgpt failed an accountin...,en,RT @moemicky18: So chatgpt failed an accountin...,RT @moemicky18 So chatgpt failed an accounting...
3,RT @JeanPROFITbiz: Les gens gagnent des millie...,fr,RT @JeanPROFITbiz: People are making thousands...,RT @JeanPROFITbiz People are making thousands ...
4,"does chatgpt mean I, as a wet lab biologist wh...",en,"does chatgpt mean I, as a wet lab biologist wh...",does chatgpt mean I as a wet lab biologist who...


#### 4. Cambiar texto a minúsculas

In [19]:
df_texto['text_clean'] = df_texto['text_clean'].apply((lambda x: x.lower()))
df_texto.head()

,text,idioma,text_ingles,text_clean
0,RT @gdb: ChatGPT as a tool for Congress: https...,en,RT @gdb: ChatGPT as a tool for Congress: https...,rt @gdb chatgpt as a tool for congress httpstc...
1,RT @gdb: ChatGPT as a tool for Congress: https...,en,RT @gdb: ChatGPT as a tool for Congress: https...,rt @gdb chatgpt as a tool for congress httpstc...
2,RT @moemicky18: So chatgpt failed an accountin...,en,RT @moemicky18: So chatgpt failed an accountin...,rt @moemicky18 so chatgpt failed an accounting...
3,RT @JeanPROFITbiz: Les gens gagnent des millie...,fr,RT @JeanPROFITbiz: People are making thousands...,rt @jeanprofitbiz people are making thousands ...
4,"does chatgpt mean I, as a wet lab biologist wh...",en,"does chatgpt mean I, as a wet lab biologist wh...",does chatgpt mean i as a wet lab biologist who...


#### 5. Tokenización

In [20]:
df_texto['text_clean'] = df_texto['text_clean'].apply(lambda x: x.rsplit())
df_texto.head()

,text,idioma,text_ingles,text_clean
0,RT @gdb: ChatGPT as a tool for Congress: https...,en,RT @gdb: ChatGPT as a tool for Congress: https...,"[rt, @gdb, chatgpt, as, a, tool, for, congress..."
1,RT @gdb: ChatGPT as a tool for Congress: https...,en,RT @gdb: ChatGPT as a tool for Congress: https...,"[rt, @gdb, chatgpt, as, a, tool, for, congress..."
2,RT @moemicky18: So chatgpt failed an accountin...,en,RT @moemicky18: So chatgpt failed an accountin...,"[rt, @moemicky18, so, chatgpt, failed, an, acc..."
3,RT @JeanPROFITbiz: Les gens gagnent des millie...,fr,RT @JeanPROFITbiz: People are making thousands...,"[rt, @jeanprofitbiz, people, are, making, thou..."
4,"does chatgpt mean I, as a wet lab biologist wh...",en,"does chatgpt mean I, as a wet lab biologist wh...","[does, chatgpt, mean, i, as, a, wet, lab, biol..."


#### 6. Eliminar stopwords

In [21]:
stopwords = nltk.corpus.stopwords.words('english')

print("Número de stopwords en de NLTK library:", len(stopwords))
stopwords[:15]

Número de stopwords en de NLTK library: 179


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours']

In [22]:
# función para remover stopwords del texto tokenizado
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

df_texto['text_clean'] = df_texto['text_clean'].apply(lambda x:remove_stopwords(x))
df_texto.head()

,text,idioma,text_ingles,text_clean
0,RT @gdb: ChatGPT as a tool for Congress: https...,en,RT @gdb: ChatGPT as a tool for Congress: https...,"[rt, @gdb, chatgpt, tool, congress, httpstcohl..."
1,RT @gdb: ChatGPT as a tool for Congress: https...,en,RT @gdb: ChatGPT as a tool for Congress: https...,"[rt, @gdb, chatgpt, tool, congress, httpstcohl..."
2,RT @moemicky18: So chatgpt failed an accountin...,en,RT @moemicky18: So chatgpt failed an accountin...,"[rt, @moemicky18, chatgpt, failed, accounting,..."
3,RT @JeanPROFITbiz: Les gens gagnent des millie...,fr,RT @JeanPROFITbiz: People are making thousands...,"[rt, @jeanprofitbiz, people, making, thousands..."
4,"does chatgpt mean I, as a wet lab biologist wh...",en,"does chatgpt mean I, as a wet lab biologist wh...","[chatgpt, mean, wet, lab, biologist, frankenst..."


#### 7. Lematización

In [23]:
wordnet_lemmatizer = nltk.stem.WordNetLemmatizer()

# función para lematizar
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

df_texto['text_clean'] = df_texto['text_clean'].apply(lambda x:lemmatizer(x))
df_texto.head()

,text,idioma,text_ingles,text_clean
0,RT @gdb: ChatGPT as a tool for Congress: https...,en,RT @gdb: ChatGPT as a tool for Congress: https...,"[rt, @gdb, chatgpt, tool, congress, httpstcohl..."
1,RT @gdb: ChatGPT as a tool for Congress: https...,en,RT @gdb: ChatGPT as a tool for Congress: https...,"[rt, @gdb, chatgpt, tool, congress, httpstcohl..."
2,RT @moemicky18: So chatgpt failed an accountin...,en,RT @moemicky18: So chatgpt failed an accountin...,"[rt, @moemicky18, chatgpt, failed, accounting,..."
3,RT @JeanPROFITbiz: Les gens gagnent des millie...,fr,RT @JeanPROFITbiz: People are making thousands...,"[rt, @jeanprofitbiz, people, making, thousand,..."
4,"does chatgpt mean I, as a wet lab biologist wh...",en,"does chatgpt mean I, as a wet lab biologist wh...","[chatgpt, mean, wet, lab, biologist, frankenst..."


#### 8. Eliminar URL's

In [24]:
# función para eliminar URL's
def erase_urls(text):
    output = [i for i in text if i.rfind("http")==-1]
    return output

df_texto['text_clean'] = df_texto['text_clean'].apply(lambda x:erase_urls(x))
df_texto.head()

,text,idioma,text_ingles,text_clean
0,RT @gdb: ChatGPT as a tool for Congress: https...,en,RT @gdb: ChatGPT as a tool for Congress: https...,"[rt, @gdb, chatgpt, tool, congress]"
1,RT @gdb: ChatGPT as a tool for Congress: https...,en,RT @gdb: ChatGPT as a tool for Congress: https...,"[rt, @gdb, chatgpt, tool, congress]"
2,RT @moemicky18: So chatgpt failed an accountin...,en,RT @moemicky18: So chatgpt failed an accountin...,"[rt, @moemicky18, chatgpt, failed, accounting,..."
3,RT @JeanPROFITbiz: Les gens gagnent des millie...,fr,RT @JeanPROFITbiz: People are making thousands...,"[rt, @jeanprofitbiz, people, making, thousand,..."
4,"does chatgpt mean I, as a wet lab biologist wh...",en,"does chatgpt mean I, as a wet lab biologist wh...","[chatgpt, mean, wet, lab, biologist, frankenst..."


### Resultado final preprocesamiento

In [25]:
df_texto[['text','text_clean']]

,text,text_clean
0,RT @gdb: ChatGPT as a tool for Congress: https...,"[rt, @gdb, chatgpt, tool, congress]"
1,RT @gdb: ChatGPT as a tool for Congress: https...,"[rt, @gdb, chatgpt, tool, congress]"
2,RT @moemicky18: So chatgpt failed an accountin...,"[rt, @moemicky18, chatgpt, failed, accounting,..."
3,RT @JeanPROFITbiz: Les gens gagnent des millie...,"[rt, @jeanprofitbiz, people, making, thousand,..."
4,"does chatgpt mean I, as a wet lab biologist wh...","[chatgpt, mean, wet, lab, biologist, frankenst..."
...,...,...
2995,RT @arabtribunenews: د. فراس الحربي يكتب: ثورة...,"[rt, @arabtribunenews, dr, firas, alharbi, wri..."
2996,I actually had to redo this one twice because ...,"[actually, redo, one, twice, chatgpt, kept, gi..."
2997,RT @rodtrent: Can ChatGPT work with your enter...,"[rt, @rodtrent, chatgpt, work, enterprise, dat..."
2998,RT @rowancheung: 2. Boston Dynamics puts ChatG...,"[rt, @rowancheung, 2, boston, dynamic, put, ch..."


### Fuentes:

- https://www.ibidem-translations.com/edu/traduccion-idiomas-nlp/#:~:text=La%20función%20“detectar”%20y%20“traducir”%20de%20Python&text=Luego%20detecta%20el%20idioma%20del,proporcionado%20al%20idioma%20de%20destino.
- https://stackoverflow.com/questions/74241589/i-keep-getting-the-error-google-trans-new-google-trans-new-google-new-transerror